In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from skopt import gp_minimize
from skopt.space import Real, Integer
from skopt.utils import use_named_args

# Seed for reproducibility
np.random.seed(42)

# Constants based on SmallSat paper
BASELINE_LATENCY = 1.42
BASELINE_FAULT_RATE = 0.33

# Parameter space for optimization
space = [
    Real(0.4, 1.5, name="timeout"),       # seconds
    Integer(20, 60, name="buffer_size")   # KB
]

# Logs
latency_log = []
fault_log = []


In [ ]:
@use_named_args(space)
def performance_fn(timeout, buffer_size):
    """
    Simulate latency and fault likelihood based on timeout and buffer.
    """
    # Simulated latency model
    latency = BASELINE_LATENCY - (timeout * 0.3) - (buffer_size * 0.005)
    latency += np.random.normal(0, 0.03)

    # Fault chance model (empirically tuned)
    fault_chance = 0.20 + (0.5 - timeout) * 0.10 + (30 - buffer_size) * 0.0025
    fault_chance = np.clip(fault_chance, 0, 1)
    fault = np.random.rand() < fault_chance

    # Log for later analysis
    latency_log.append(latency)
    fault_log.append(1 if fault else 0)

    # Penalize faults in optimization
    return latency + (0.7 if fault else 0)


In [ ]:
result = gp_minimize(
    func=performance_fn,
    dimensions=space,
    acq_func="EI",     # Expected Improvement
    n_calls=30,        # Matches SmallSat paper
    random_state=42
)

best_timeout = result.x[0]
best_buffer = result.x[1]

print(" Best Configuration Found:")
print(f"Timeout     = {best_timeout:.2f} s")
print(f"Buffer Size = {best_buffer} KB")


In [ ]:
avg_latency = np.mean(latency_log)
fault_rate = np.sum(fault_log) / len(fault_log)

print(f"\n Average Latency: {avg_latency:.2f} s")
print(f"  Fault Rate: {fault_rate * 100:.2f} %")
print("\n Note: Target fault rate from SmallSat paper = ~16.67% over 30 runs.")


In [ ]:
plt.figure(figsize=(10, 4))
plt.plot(latency_log, label="Latency per Cycle", marker='o')
plt.axhline(BASELINE_LATENCY, color="red", linestyle="--", label="Baseline (1.42s)")
plt.axhline(0.89, color="green", linestyle=":", label="Target (0.89s)")
plt.title("Latency Optimization Over 30 Simulation Cycles")
plt.xlabel("Cycle")
plt.ylabel("Latency (s)")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()


In [ ]:
plt.figure(figsize=(10, 4))
cumulative_rate = np.cumsum(fault_log) / np.arange(1, len(fault_log) + 1)
plt.plot(cumulative_rate, label="Cumulative Fault Rate", color="orange", marker="x")
plt.axhline(BASELINE_FAULT_RATE, color="red", linestyle="--", label="Baseline (33%)")
plt.axhline(0.166, color="green", linestyle=":", label="Target (16.6%)")
plt.title("Fault Rate Reduction Over 30 Simulation Cycles")
plt.xlabel("Cycle")
plt.ylabel("Fault Rate")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()


Note: Target fault rate from SmallSat paper = ~16.67% over 30 runs.


Note: Fault rate here is based on 30 simulation cycles and may vary run to run.
The SmallSat paper reports an **average fault rate of ~16.67%** across multiple testbeds.
